In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
80,Every once in a long while a movie will come a...,negative
980,"Like his early masterpiece ""The Elephant Man"" ...",positive
309,"I love these ""Diaper Baby"" movies! You couldn'...",positive
362,Steve Biko was a black activist who tried to r...,positive
407,"THE RUNNING MAN, along with TOTAL RECALL, is m...",positive


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
80,every long movie come along awful feel compell...,negative
980,like early masterpiece the elephant man lynch ...,positive
309,love diaper baby movie make movie like today r...,positive
362,steve biko black activist tried resist white m...,positive
407,running man along total recall favorite schwar...,positive


In [6]:
df['sentiment'].value_counts()

sentiment
negative    256
positive    244
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
80,every long movie come along awful feel compell...,0
980,like early masterpiece the elephant man lynch ...,1
309,love diaper baby movie make movie like today r...,1
362,steve biko black activist tried resist white m...,1
407,running man along total recall favorite schwar...,1


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=200)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [16]:
#  Here we are setting our remote tracking server

import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Rahul-Dewangan/MLOps-Capstone-Project.mlflow')
dagshub.init(repo_owner='Rahul-Dewangan', repo_name='MLOps-Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2026-03-01 17:17:13,301 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/Rahul-Dewangan/MLOps-Capstone-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Rahul-Dewangan/MLOps-Capstone-Project"

2026-03-01 17:17:13,316 - INFO - Initialized MLflow to track repo "Rahul-Dewangan/MLOps-Capstone-Project"


Repository Rahul-Dewangan/MLOps-Capstone-Project initialized!

2026-03-01 17:17:13,325 - INFO - Repository Rahul-Dewangan/MLOps-Capstone-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/84676e18e81842b19bbf657ef7b368c4', creation_time=1772365067657, experiment_id='0', last_update_time=1772365067657, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}, workspace='default'>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 200)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-03-01 17:17:29,828 - INFO - Starting MLflow run...
2026-03-01 17:17:30,350 - INFO - Logging preprocessing parameters...
2026-03-01 17:17:31,395 - INFO - Initializing Logistic Regression model...
2026-03-01 17:17:31,397 - INFO - Fitting the model...
2026-03-01 17:17:31,483 - INFO - Model training complete.
2026-03-01 17:17:31,485 - INFO - Logging model parameters...
2026-03-01 17:17:31,798 - INFO - Making predictions...
2026-03-01 17:17:31,798 - INFO - Calculating evaluation metrics...
2026-03-01 17:17:31,831 - INFO - Logging evaluation metrics...
2026-03-01 17:17:33,096 - INFO - Saving and logging the model...
2026/03/01 17:17:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/03/01 17:17:37 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The rec

🏃 View run charming-flea-473 at: https://dagshub.com/Rahul-Dewangan/MLOps-Capstone-Project.mlflow/#/experiments/0/runs/c60b57a1e43d4f519a150bc7422a05b4
🧪 View experiment at: https://dagshub.com/Rahul-Dewangan/MLOps-Capstone-Project.mlflow/#/experiments/0
